In [1]:
import numpy as np
import wandb
import torch
import pcl
import open3d as o3d
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
filepath = "/home/server/PIN_SLAM/data/wudasuidao/normal/30frame/1.ply"
pc_load = o3d.io.read_point_cloud(filepath)
# origin_points = np.asarray(pc_load.points)

# pc_load.paint_uniform_color([0, 0, 1.0])  # 渲染颜色
point1 = np.asarray(pc_load.points)
normals = np.asarray(pc_load.normals)


# normals = np.asarray(pc_load.normals)

o3d.visualization.draw_geometries([pc_load],
                                # zoom=0.3412,
                                # front=[0.4257, -0.2125, -0.8795],
                                # lookat=[2.6172, 2.0475, 1.532],
                                # up=[-0.0694, -0.9768, 0.2024],
                                point_show_normal=True
                                )

In [ ]:
# 剔除近邻点少的点，相当于作了一次radius剔除
# parameters
k_out = 30
radius_out = 3
pc_load, ind = pc_load.remove_radius_outlier(nb_points=k_out, radius=radius_out) # cl是pointcloud ind是剩下的点的索引
# pc_load, idx = pc_load.remove_statistical_outlier(nb_neighbors=k, std_ratio=3, print_progress=False)

k = 20
radius = 0.3
pc_load.estimate_normals(
    # search_param=o3d.geometry.KDTreeSearchParamHybrid(radius = radius, max_nn = k)
    search_param=o3d.geometry.KDTreeSearchParamKNN(knn = k)
)
# pc_load.orient_normals_towards_camera_location(camera_location=np.array([0.0, 0.0, 0.0]))
neighbors_normal_index={}

## neighbors_normal_index
kdtree = o3d.geometry.KDTreeFlann(pc_load)

T1 = time.time()
origin_points = np.asarray(pc_load.points)

# Only costs 1.11s-> 0.3s
for i in range(len(pc_load.points)):
    [a,idx,b] = kdtree.search_knn_vector_3d(pc_load.points[i], k)
    neighbors_normal_index[i] = idx

# kk = neighbors_normal_index[1][0]

T2 = time.time()
print(f"Time cost:{T2-T1}")

normals = np.asarray(pc_load.normals)
norms = np.linalg.norm(normals, axis=1, keepdims=True)
# 归一化
normals = normals / norms
old_normals = np.copy(normals)

# pc_load.orient_normals_towards_camera_location()
pc_load.orient_normals_consistent_tangent_plane(100)


o3d.visualization.draw_geometries([pc_load],
                                # zoom=0.3412,
                                # front=[0.4257, -0.2125, -0.8795],
                                # lookat=[2.6172, 2.0475, 1.532],
                                # up=[-0.0694, -0.9768, 0.2024],
                                point_show_normal=True)